In [1]:
import numpy as np
import itertools
from scipy.misc import comb
from decimal import Decimal as D
from decimal import getcontext
import math 
getcontext().prec = 40

$$H(p) = \frac13 \log_23 + \frac23 \log_21.5 $$

In [2]:
def log2(x):
    return D(math.log2(x))

def dot(x, y):
    return sum(x_ * y_ for x_, y_ in zip(x, y))

def entropy(probs):
    log_probs = [D(log2(1 / p)) for p in probs]
    return dot(probs, log_probs)

In [3]:
n = 5
# probs = [1/3, 2/3]
probs = [D(1 / 3), D(2 / 3)]
log_probs = [log2(p) for p in probs]

H = entropy(probs)

print("entropy = %.4f" % H)

entropy = 0.9183


In [5]:
def outcomes(length):
    """ All possible sequencies of 0 and 1 
    """
    lenght = int(length)
    return (
        tuple(int(c) for c in format(i, 'b').zfill(length))
        for i in range(2 ** lenght)
    )

def outcomes_lazy(length):
    """ All possible combination with replacement of 0 and 1 
    with number of possible permuntations
    """
    length = int(length)
    return map(lambda x: (x, comb(length, sum(x), exact=True)),
        itertools.combinations_with_replacement((0, 1), length)
    )

def prob(outcome):
    nones = sum(outcome)
    n = len(outcome)
    return probs[0] ** (n - nones) * probs[1] ** nones

def log_prob(outcome):
    nones = sum(outcome)
    n = len(outcome)
    return dot(log_probs, [n - nones, nones]) 

def n_entropy(n):
    return -sum(
        c * prob(outcome) * log_prob(outcome)
        for outcome, c in outcomes_lazy(n)
    )

In [6]:
# testing 
assert abs(n_entropy(1) - entropy(probs)) < 1e-10
assert abs(n_entropy(10) - entropy([prob(outcome) for outcome in outcomes(10)])) < 1e-10

for m in [2, 5, 10]:
    outcomes_prob = sum(prob(outcome) for outcome in outcomes(m))
    assert abs(outcomes_prob - 1) < 1e-10, outcomes_prob

for m in [2, 5, 10, 100, 1000]:
    outcomes_prob = sum(prob(outcome) * c for outcome, c in outcomes_lazy(m))
    assert abs(outcomes_prob - 1) < 1e-10, outcomes_prob

In [7]:
n = 5
eps = D(0.138)

min_p = 2 ** (-n * (H + eps))
max_p = 2 ** (-n * (H - eps))

typical_seq = []
print("x\t\t| P\t | typical")
print('-' * 34)
for x in outcomes(n):
    
    prob_x = np.prod([probs[i] for i in x])
    typical = min_p < prob_x < max_p
    print("{} | {:.4f} | {}".format(x, float(prob_x), typical))
    
    if typical:
        typical_seq.append(prob_x)

print()
print("#typical =", len(typical_seq))
print('P(typical) = {:.5f}'.format(sum(typical_seq)))

x		| P	 | typical
----------------------------------
(0, 0, 0, 0, 0) | 0.0041 | False
(0, 0, 0, 0, 1) | 0.0082 | False
(0, 0, 0, 1, 0) | 0.0082 | False
(0, 0, 0, 1, 1) | 0.0165 | False
(0, 0, 1, 0, 0) | 0.0082 | False
(0, 0, 1, 0, 1) | 0.0165 | False
(0, 0, 1, 1, 0) | 0.0165 | False
(0, 0, 1, 1, 1) | 0.0329 | True
(0, 1, 0, 0, 0) | 0.0082 | False
(0, 1, 0, 0, 1) | 0.0165 | False
(0, 1, 0, 1, 0) | 0.0165 | False
(0, 1, 0, 1, 1) | 0.0329 | True
(0, 1, 1, 0, 0) | 0.0165 | False
(0, 1, 1, 0, 1) | 0.0329 | True
(0, 1, 1, 1, 0) | 0.0329 | True
(0, 1, 1, 1, 1) | 0.0658 | True
(1, 0, 0, 0, 0) | 0.0082 | False
(1, 0, 0, 0, 1) | 0.0165 | False
(1, 0, 0, 1, 0) | 0.0165 | False
(1, 0, 0, 1, 1) | 0.0329 | True
(1, 0, 1, 0, 0) | 0.0165 | False
(1, 0, 1, 0, 1) | 0.0329 | True
(1, 0, 1, 1, 0) | 0.0329 | True
(1, 0, 1, 1, 1) | 0.0658 | True
(1, 1, 0, 0, 0) | 0.0165 | False
(1, 1, 0, 0, 1) | 0.0329 | True
(1, 1, 0, 1, 0) | 0.0329 | True
(1, 1, 0, 1, 1) | 0.0658 | True
(1, 1, 1, 0, 0) | 0.0329 | True
(1,

In [8]:
min_card = (1 - eps) * 2 ** (n * (H - eps))
max_card = 2 ** (n * (H + eps))
min_card < len(typical_seq) < max_card

True

In [9]:
n_range = [10, 100, 500, 1000, 2000, 5000]
eps_range = [D(eps) for eps in [.01, .1, .2, .3, .4, .5]]

result = np.zeros((len(n_range), len(eps_range), 2))

print('n\t eps\t fr\t\t prob\t\t 3?')

for i, n in enumerate(n_range):
    
    for j, eps in enumerate(eps_range):
        typical_count = 0
        typical_prob = 0
        
        min_p = 2 ** (-n * (H + eps))
        max_p = 2 ** (-n * (H - eps))

        for x, c in outcomes_lazy(n):
            prob_x = prob(x)
#             typical = abs(log2(float(1/prob_x)) / n - H) <= eps
            typical = min_p < prob_x < max_p
    
            if typical:
                typical_count += c
                typical_prob += prob_x * c

        freq = typical_count / 2**n
        
        min_card = (1 - eps) * 2 ** (n * (H - eps))
        max_card = 2 ** (n * (H + eps))
        card = min_card < typical_count < max_card
#         card = np.log2(1 - eps) + n * (H - eps) <= np.log2(typical_count) <= n * np.log2(H + eps)
        
        print("{}\t {:.3f}\t {:.10f}\t {:.10f}\t {}".format(n, eps, freq, typical_prob, card))
        result[i][j] = [freq, typical_prob]
        if typical_prob > 1 - 1e-8:
            break
        
    print()


n	 eps	 fr		 prob		 3?
10	 0.010	 0.0000000000	 0.0000000000	 False
10	 0.100	 0.3222656250	 0.4877305289	 True
10	 0.200	 0.6123046875	 0.8193872885	 True
10	 0.300	 0.8271484375	 0.9629968331	 True
10	 0.400	 0.9453125000	 0.9965960474	 True
10	 0.500	 0.9892578125	 0.9996443632	 True

100	 0.010	 0.0006905766	 0.1675243991	 False
100	 0.100	 0.0966739247	 0.9667359488	 True
100	 0.200	 0.7579407932	 0.9999800784	 True
100	 0.300	 0.9966814397	 0.9999999996	 True

500	 0.010	 0.0000000000	 0.3647288369	 True
500	 0.100	 0.0013483886	 0.9999977537	 True
500	 0.200	 0.9300407372	 1.0000000000	 True

1000	 0.010	 0.0000000000	 0.4975730959	 True
1000	 0.100	 0.0000126719	 1.0000000000	 True

2000	 0.010	 0.0000000000	 0.6572264751	 True
2000	 0.100	 0.0000000011	 1.0000000000	 True

5000	 0.010	 0.0000000000	 0.8663954138	 True
5000	 0.100	 0.0000000000	 1.0000000000	 True



In [10]:
eps = D(.138)

for n in range(100, 5000, 50):
    typical_prob = 0
    
    for x, c in outcomes_lazy(n):
        prob_x = prob(x)
        typical = min_p < prob_x < max_p

        if typical:
            typical_prob += prob_x * c
            
    if typical_prob > 1 - eps:
        print(n)
        break

4500
